# For Testing Datasets and APIs

In [2]:
import requests

In [3]:
output = "applications/json"
r = requests.get('https://api.tfl.gov.uk/Journey/JourneyResults/51.509372,-0.076177/to/51.501105,-0.126124?nationalSearch=false&date=20250224&time=0900&timeIs=Arriving&journeyPreference=LeastWalking&alternativeCycle=false&walkingOptimization=true&routeBetweenEntrances=false', 
                 headers={'Accept': output})
r.status_code


200

nationalSearch=false
20250224 
09:00
arriving
alternativeCycle: false
walkingOptimization: true
routeBetweenEntrances: false

journeyPreference
mode
maxWalkingMinutes
walkingSpeed:  "slow" | "average" | "fast"

In [4]:
request = r.json()
request

{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.ItineraryResult, Tfl.Api.Presentation.Entities',
 'journeys': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Journey, Tfl.Api.Presentation.Entities',
   'startDateTime': '2025-02-24T08:32:00',
   'duration': 22,
   'arrivalDateTime': '2025-02-24T08:54:00',
   'alternativeRoute': False,
   'legs': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Leg, Tfl.Api.Presentation.Entities',
     'duration': 4,
     'instruction': {'$type': 'Tfl.Api.Presentation.Entities.Instruction, Tfl.Api.Presentation.Entities',
      'summary': 'Walk to Tower Hill',
      'detailed': 'Walk to Tower Hill',
      'steps': [{'$type': 'Tfl.Api.Presentation.Entities.InstructionStep, Tfl.Api.Presentation.Entities',
        'description': ' for 53 metres',
        'turnDirection': 'STRAIGHT',
        'streetName': '',
        'distance': 53,
        'cumulativeDistance': 53,
        'skyDirection': 254,
        'skyDirectionDescription': 'West

In [5]:
for journey in request['journeys']:
    print(journey['duration'])
 #[0] #[0]['duration'] / 60

22
21
21


In [6]:
# extract the public transport used in a journey
for journey in request['journeys']:
    for leg in journey['legs']:
        
        print(leg['mode']['name'])
        if leg['mode']['name'] != 'walking':
            print(leg['routeOptions'][0]['name'])
            #print(leg['mode']['name'])
    print()

walking
tube
District
walking

walking
tube
District
walking

walking
tube
District
walking



In [36]:
def get_journey(from_lat, from_lon, to_lat, to_lon):
    """
    Get the average journey time in minutes for public transport duration from one location to another, without the walking time
    """

    output = "applications/json"
    r = requests.get(f'https://api.tfl.gov.uk/Journey/JourneyResults/{from_lat},{from_lon}/to/{to_lat},{to_lon}?nationalSearch=false&date=20250224&time=0900&timeIs=Arriving&journeyPreference=LeastWalking&alternativeCycle=false&walkingOptimization=true&routeBetweenEntrances=false&app_id=Burghandi&app_key=95598b12d85e401fbe896c199885b792', 
                     headers={'Accept': output})
    if (r.status_code!=200):
        raise Exception(f'Failed to get journey: from {from_lat},{from_lon} to {to_lat},{to_lon} with status code {r.status_code}')
    
    journeys = r.json()['journeys']
    if len(journeys) < 1:
        raise Exception(f'No journey found: from {from_lat},{from_lon} to {to_lat},{to_lon}')
    
    if journeys[0] == "walking":
        journeys = journeys[1:]
    journey_duration = [journey['duration'] for journey in journeys]

    return round(sum(journey_duration)/len(journey_duration))

In [34]:
import pickle
import os
import concurrent.futures
import time

In [ ]:
def get_districts_to_work(districts, lat, lng):
    journey_times = {}

    def fetch_journey_time(data):
        try:
            time.sleep(0.5)
            return data["outcode"], get_journey(lat, lng, float(data["latitude"]), float(data["longitude"]))
            
        except Exception as e:
            print(e)
            return data["outcode"], None
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(fetch_journey_time, districts.values())
        

    for outcode, journey_time in results:
        try:
            journey_times[outcode] = journey_time
        except Exception as e:
            print(e)
            pass

    print(journey_times)

In [ ]:
districts = pickle.load(open('../flask_backend/districts.pkl', "rb"))

In [49]:
get_districts_to_work(districts, 51.509372, -0.076177)

Failed to get journey: from 51.509372,-0.076177 to 51.51050094999998,-0.08343551250000003 with status code 404
{'BR1': 66, 'BR2': 62, 'BR3': 73, 'BR4': 73, 'BR5': 71, 'BR6': 66, 'BR7': 80, 'BR8': 73, 'CM13': 84, 'CM14': 71, 'CM23': 70, 'CR0': 47, 'CR2': 71, 'CR3': 69, 'CR4': 57, 'CR5': 56, 'CR6': 69, 'CR7': 59, 'CR8': 67, 'CR9': 43, 'CR90': 59, 'DA1': 77, 'DA14': 79, 'DA15': 70, 'DA16': 57, 'DA17': 56, 'DA18': 46, 'DA5': 62, 'DA6': 65, 'DA7': 58, 'DA8': 88, 'E1': 17, 'E10': 37, 'E11': 34, 'E12': 43, 'E13': 30, 'E14': 20, 'E15': 27, 'E16': 31, 'E17': 45, 'E18': 37, 'E1W': 17, 'E2': 28, 'E20': 31, 'E3': 20, 'E4': 70, 'E5': 36, 'E6': 41, 'E7': 31, 'E8': 32, 'E9': 41, 'E98': 11, 'EC1A': 18, 'EC1M': 21, 'EC1N': 26, 'EC1P': 27, 'EC1R': 28, 'EC1V': 36, 'EC1Y': 31, 'EC2A': 24, 'EC2M': 19, 'EC2N': 19, 'EC2P': 27, 'EC2R': 23, 'EC2V': 24, 'EC2Y': 25, 'EC3A': 13, 'EC3M': 11, 'EC3N': 11, 'EC3P': 24, 'EC3R': None, 'EC3V': 14, 'EC4A': 21, 'EC4M': 20, 'EC4N': 16, 'EC4P': 27, 'EC4R': 11, 'EC4V': 18, 'E

In [39]:
with open('../flask_backend/districts.pkl', 'rb') as f:
    districts = pickle.load(f)
    journey_times = {}

    def fetch_journey_time(data):
        try:
            #time.sleep(0.5)
            return data["outcode"], get_journey(51.509372, -0.076177, float(data["latitude"]), float(data["longitude"]))
            
        except Exception as e:
            print(e)
            return data["outcode"], None
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(fetch_journey_time, districts.values())
        

    for outcode, journey_time in results:
        try:
            journey_times[outcode] = journey_time
        except Exception as e:
            print(e)
            pass

    print(journey_times)

Failed to get journey: from 51.509372,-0.076177 to 51.51050094999998,-0.08343551250000003 with status code 404
{'BR1': 66, 'BR2': 62, 'BR3': 73, 'BR4': 73, 'BR5': 71, 'BR6': 66, 'BR7': 80, 'BR8': 73, 'CM13': 84, 'CM14': 71, 'CM23': 70, 'CR0': 47, 'CR2': 71, 'CR3': 69, 'CR4': 57, 'CR5': 56, 'CR6': 69, 'CR7': 59, 'CR8': 67, 'CR9': 43, 'CR90': 59, 'DA1': 77, 'DA14': 79, 'DA15': 70, 'DA16': 57, 'DA17': 56, 'DA18': 46, 'DA5': 62, 'DA6': 65, 'DA7': 58, 'DA8': 88, 'E1': 17, 'E10': 37, 'E11': 34, 'E12': 43, 'E13': 30, 'E14': 20, 'E15': 27, 'E16': 31, 'E17': 45, 'E18': 37, 'E1W': 17, 'E2': 28, 'E20': 31, 'E3': 20, 'E4': 70, 'E5': 36, 'E6': 41, 'E7': 31, 'E8': 32, 'E9': 41, 'E98': 11, 'EC1A': 18, 'EC1M': 21, 'EC1N': 26, 'EC1P': 27, 'EC1R': 28, 'EC1V': 36, 'EC1Y': 31, 'EC2A': 24, 'EC2M': 19, 'EC2N': 19, 'EC2P': 27, 'EC2R': 23, 'EC2V': 24, 'EC2Y': 25, 'EC3A': 13, 'EC3M': 11, 'EC3N': 11, 'EC3P': 24, 'EC3R': None, 'EC3V': 14, 'EC4A': 21, 'EC4M': 20, 'EC4N': 16, 'EC4P': 27, 'EC4R': 11, 'EC4V': 18, 'E

In [ ]:
get_journey(51.509372,-0.076177, 51.501105,-0.126124)

21

In [27]:
print(journey_times)

{'outcode': 84}
